# OpenVaccine: COVID-19 mRNA Vaccine Degradation Prediction Getting Started Notebook

A starter notebook for those planning to build your submission from scratch. Will update with EDA as and when time permits.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import json
from pandas.io.json import json_normalize

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Read train and test json

In [4]:
train = pd.read_json('../train.json', lines=True)
train.T

,0,1,2,3,4,5,6,7,8,9,...,2390,2391,2392,2393,2394,2395,2396,2397,2398,2399
SN_filter,1,0,1,0,0,1,1,1,0,1,...,0,1,1,1,0,1,1,0,0,1
deg_50C,"[0.6382, 3.4773, 0.9988, 1.3228, 0.78770000000...","[7.6692, 0.0, 10.9561, 0.0, 0.0, 0.0, 0.0, 0.0...","[0.9501000000000001, 1.7974999999999999, 1.499...","[7.6692, -1.3223, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[0.0, 5.1198, -0.3551, -0.3518, 0.0, 0.0, 0.0,...","[0.5882000000000001, 1.1786, 0.9704, 0.6035, 0...","[2.0964, 3.3688000000000002, 0.6399, 2.1053, 1...","[1.2852000000000001, 2.5460000000000003, 0.234...","[-0.5478000000000001, 0.6198, -0.2701, 1.7043,...","[1.0121, 2.0629, 1.3669, 0.9423, 0.88040000000...",...,"[0.6438, 2.0112, 0.4963, 1.7001, 1.0192, 0.642...","[0.8335, 3.5268, 1.7208999999999999, 0.3437, 1...","[0.3635, 2.0364, 1.1619, 0.5933, 0.561, 0.091,...","[0.8569, 3.0359, 0.28350000000000003, 1.3674, ...","[0.2897, 1.1666, 0.135, 0.4742, 0.9522, 0.8408...","[0.5759000000000001, 2.3736, 1.4158, 0.1914000...","[0.6526000000000001, 0.2548, 0.6927, 0.9316000...","[2.9632, 6.3911999999999995, 1.5339, 1.4749, 1...","[0.5814, 1.5119, 1.1749, 1.2676, 0.22190000000...","[0.7043, 1.4864, 1.3035, 1.2176, 0.82900000000..."
deg_Mg_50C,"[0.35810000000000003, 2.9683, 0.2589, 1.4552, ...","[4.8511, 4.0426, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.5163, 1.6823000000000001, 1.0426, 0.7902, 0...","[0.0, -0.8365, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","[2.2052, 1.7947000000000002, 0.7457, 3.1233, 0...","[0.22460000000000002, 1.7281, 1.381, 0.6623, 0...","[1.073, 2.8604000000000003, 1.9936, 1.0273, 1....","[0.49060000000000004, 4.6339, 1.95860000000000...","[2.0792, 5.0599, 2.3826, 0.8223, 0.0, 0.0, 0.4...","[0.23140000000000002, 1.1193, 1.103, 0.5249, 0...",...,"[0.2846, 2.1795, 0.8958, 1.0863, 0.6933, 0.591...","[0.41500000000000004, 4.1486, 2.1685, 0.5299, ...","[0.6743, 4.0084, 0.8681000000000001, 0.419, 0....","[0.594, 4.0831, 0.5513, 1.8365, 1.022, 1.3291,...","[0.2661, 0.5771000000000001, 0.3517, 0.295, 0....","[0.47900000000000004, 1.9583, 2.4635, 0.0512, ...","[0.2964, 0.9351, 0.2555, 0.7603000000000001, 0...","[0.6038, 3.0322, 0.9703, 0.4756, 0.9154, 0.0, ...","[0.3262, 1.3932, 0.8832000000000001, 0.8144, 0...","[1.3281, 2.3854, 2.0464, 1.2384, 0.631, 0.1848..."
deg_Mg_pH10,"[0.7556, 2.983, 0.2526, 1.3789, 0.637600000000...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.2504, 1.4021, 0.9804, 0.49670000000000003, ...","[0.0, -0.5083, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","[2.1058, 3.138, 2.5437000000000003, 1.0932, 0....","[0.9559000000000001, 1.9442, 1.0114, 0.5105000...","[0.46040000000000003, 3.6695, 0.78550000000000...","[1.6912, 5.2652, 2.3901, 0.45890000000000003, ...","[1.1295, 1.1239, 1.0301, 0.0, 0.71910000000000...","[0.6237, 1.0674, 1.2511, 0.30560000000000004, ...",...,"[0.7311000000000001, 2.5771, 0.1733, 0.7628, 0...","[1.0707, 4.7169, 1.8673000000000002, 0.3352, 2...","[0.6185, 3.9969, 0.7495, 0.28340000000000004, ...","[0.7334, 3.4708, 0.1658, 1.1059, 0.7204, 1.265...","[0.6612, 1.0221, 0.1676, 0.1648, 0.5634, 0.645...","[0.6439, 2.0117, 1.3682, 0.0918, 0.65860000000...","[0.3619, 0.6924, 0.2988, 0.3639, 0.545, 0.2263...","[1.7708, 0.67, 1.7689, 1.0919, 0.0, 0.0, 0.0, ...","[0.4544, 2.4603, 0.8778, 0.6402, 0.28340000000...","[1.4842, 2.4813, 1.737, 1.2082, 0.959000000000..."
deg_error_50C,"[0.2167, 0.34750000000000003, 0.188, 0.2124, 0...","[15.4857, 7.9596, 13.3957, 5.8777, 5.8777, 5.8...","[0.14980000000000002, 0.1761, 0.1517, 0.116700...","[15.3995, 8.1124, 7.7824, 7.7824, 7.7824, 7.78...","[1.3285, 3.6173, 1.3057, 1.3021, 1.1507, 1.150...","[0.2864, 0.24710000000000001, 0.2222, 0.1903, ...","[0.9306000000000001, 1.0496, 0.5844, 0.7796000...","[0.7429, 0.9137000000000001, 0.480400000000000...","[1.1274, 1.8647, 1.0426, 2.075, 1.689, 1.654, ...","[0.2645, 0.3039, 0.2335, 0.19440000000000002, ...",...,"[0.3372, 0.453, 0.2574, 0.3629, 0.284000000000...","[0.21660000000000001, 0.3588, 0.2356, 0.122, 0...","[0.2671, 0.42010000000000003, 0.294, 0.222, 0....","[0.3397, 0.48

Not so intuitive, right. Lets see if we can flatten it.

One thing to note is that for each parameter(5 nos) that is to be output by the model, two lists each of size seq_scored is provided for the parameter and the error in that parameter.

In [7]:
train_json = [json.loads(line) for line in open('../train.json', 'r')]

AttributeError: 'dict' object has no attribute 'loads'

In [6]:
for json in train_json:
    json['step'] = list(range(json['seq_scored']))
    json['sequence'] = list(json['sequence'])
    json['structure'] = list(json['structure'])
    json['predicted_loop_type'] = list(json['predicted_loop_type'])
    json['unpaired_probability'] = list(sum(np.load('../bpps/'+json['id']+'.npy')))

In [8]:
train_json[0]

{'index': 0,
 'id': 'id_001f94081',
 'sequence': ['G',
  'G',
  'A',
  'A',
  'A',
  'A',
  'G',
  'C',
  'U',
  'C',
  'U',
  'A',
  'A',
  'U',
  'A',
  'A',
  'C',
  'A',
  'G',
  'G',
  'A',
  'G',
  'A',
  'C',
  'U',
  'A',
  'G',
  'G',
  'A',
  'C',
  'U',
  'A',
  'C',
  'G',
  'U',
  'A',
  'U',
  'U',
  'U',
  'C',
  'U',
  'A',
  'G',
  'G',
  'U',
  'A',
  'A',
  'C',
  'U',
  'G',
  'G',
  'A',
  'A',
  'U',
  'A',
  'A',
  'C',
  'C',
  'C',
  'A',
  'U',
  'A',
  'C',
  'C',
  'A',
  'G',
  'C',
  'A',
  'G',
  'U',
  'U',
  'A',
  'G',
  'A',
  'G',
  'U',
  'U',
  'C',
  'G',
  'C',
  'U',
  'C',
  'U',
  'A',
  'A',
  'C',
  'A',
  'A',
  'A',
  'A',
  'G',
  'A',
  'A',
  'A',
  'C',
  'A',
  'A',
  'C',
  'A',
  'A',
  'C',
  'A',
  'A',
  'C',
  'A',
  'A',
  'C'],
 'structure': ['.',
  '.',
  '.',
  '.',
  '.',
  '(',
  '(',
  '(',
  '(',
  '(',
  '(',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  ')',
  ')',
  ')',
  ')',
  '.',
  ')',
  ')',
  '.',
  '(',


In [9]:
train = pd.json_normalize(data = train_json, 
                            record_path ='reactivity',  
                            meta =['id','signal_to_noise',
                                  'SN_filter','seq_length','seq_scored']) 
train.rename(columns={0:'reactivity'}, inplace=True)
train['step'] = pd.json_normalize(data = train_json, 
                            record_path ='step'
                                        )
train['unpaired_probability'] = pd.json_normalize(data = train_json, 
                            record_path ='unpaired_probability'
                                        )
train['sequence'] = pd.json_normalize(data = train_json, 
                            record_path ='sequence'
                                        )
train['structure'] = pd.json_normalize(data = train_json, 
                            record_path ='structure'
                                        )
train['predicted_loop_type'] = pd.json_normalize(data = train_json, 
                            record_path ='predicted_loop_type'
                                        )
train['reactivity_error'] = pd.json_normalize(data = train_json, 
                            record_path ='reactivity_error'
                                        )
train['deg_Mg_pH10'] = pd.json_normalize(data = train_json, 
                            record_path ='deg_Mg_pH10'
                                        )
train['deg_error_Mg_pH10'] = pd.json_normalize(data = train_json, 
                            record_path ='deg_error_Mg_pH10'
                                        )
train['deg_pH10'] = pd.json_normalize(data = train_json, 
                            record_path ='deg_pH10',
                                        )
train['deg_error_pH10'] = pd.json_normalize(data = train_json, 
                            record_path ='deg_error_pH10',
                                        )
train['deg_Mg_50C'] = pd.json_normalize(data = train_json, 
                            record_path ='deg_Mg_50C',
                                        )
train['deg_error_Mg_50C'] = pd.json_normalize(data = train_json, 
                            record_path ='deg_error_Mg_50C',
                                        )
train['deg_50C'] = pd.json_normalize(data = train_json, 
                            record_path ='deg_50C',
                                        )
train['deg_error_50C'] = pd.json_normalize(data = train_json, 
                            record_path ='deg_error_50C',
                                        )

train.set_index(['id','step'], inplace=True)
train.T

AttributeError: module 'pandas' has no attribute 'json_normalize'

In [10]:
train.describe()

,SN_filter,index,seq_length,seq_scored,signal_to_noise
count,2400.000000,2400.000000,2400.0,2400.0,2400.000000
mean,0.662083,1199.500000,107.0,68.0,4.530456
std,0.473099,692.964646,0.0,0.0,2.835142
min,0.000000,0.000000,107.0,68.0,-0.103000
25%,0.000000,599.750000,107.0,68.0,2.391000
50%,1.000000,1199.500000,107.0,68.0,4.442500
75%,1.000000,1799.250000,107.0,68.0,6.294250
max,1.000000,2399.000000,107.0,68.0,17.194000


Repeat the same with test set

In [ ]:
test = pd.read_json('../test.json', lines=True)
test.T

In [ ]:
test.describe()

### Make dummy submission File

This will be overwritten if everything works fine.

In [ ]:
submission = pd.read_csv('../sample_submission.csv', index_col= 0)
submission.to_csv('submission.csv')
submission.T

You have to make 107 to 130 predictions per input for the 5 parameters (only 3 of these are used for scoring).

## Develop Your model

Seems you will need something like an LSTM that works on sequence data.

In [ ]:
# Dummy

## Train your model

In [ ]:
# Dummy

## Perform your prediction

In [ ]:
# Dummy

## Filling dummy values

Lets use values mean and std from train data as a dummy submission

In [ ]:
train.describe()

In [ ]:
n = submission.shape[0]
reactivity_bar = 0.374922; reactivity_sd = 0.725652
deg_Mg_pH10_bar = 0.446303; deg_Mg_pH10_sd = 0.704172
deg_pH10_bar = 0.446911; deg_pH10_sd = 1.285747
deg_Mg_50C_bar = 0.407030; deg_Mg_50C_sd = 0.868013
deg_50C_bar = 0.425889; deg_50C_sd = 1.122356
submission['reactivity']=np.random.normal(reactivity_bar, reactivity_sd, size=n)
submission['deg_Mg_pH10']=np.random.normal(deg_Mg_pH10_bar, deg_Mg_pH10_sd, size=n)
submission['deg_pH10']=np.random.normal(deg_pH10_bar, deg_pH10_sd, size=n)
submission['deg_Mg_50C']=np.random.normal(deg_Mg_50C_bar, deg_Mg_50C_sd, size=n)
submission['deg_50C']=np.random.normal(deg_50C_bar, deg_50C_sd, size=n)
submission

## Submit

In [ ]:
submission.to_csv('submission.csv')

# EOF